# クロードを使った関数呼び出し

クロードにいくつかの関数の呼び出し方を教えてみましょう！

これをいくつかの段階で行います。
1. クロードに関数の説明と使い方を説明する。
2. クロードにその関数の呼び出しが必要な何かをするように指示する。
3. クロードが関数を呼び出したら、応答を一時停止し、コードで関数を呼び出す。
4. もう一度クロードを呼び出し、コードから返された値を渡す。
5. ユーザーに最終的な応答を返す。

注：https://github.com/anthropics/anthropic-tools で、より自動化された関数呼び出しのリポジトリを見ることができます。このノートブックは、そのリポジトリの仕組みを詳しく説明することを目的としています。さらに、全体的な関数呼び出しのパフォーマンスと利便性はすぐに向上すると期待しています。

関数呼び出しは、大規模言語モデルの制限を回避するのに役立ちます。その1つが大きな数の掛け算です。これを実現する「電卓」関数を実装してみましょう。

In [3]:
from anthropic import Anthropic
import re 
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

まず、クロードのデフォルトの動作を見てみましょう。

In [4]:
multiplication_message = {
 "role": "user",
 "content": "1,984,135と9,343,116をかけ算してください。"
}

message = client.messages.create(
 model=MODEL_NAME, 
 max_tokens=1024,
 messages=[multiplication_message] 
).content[0].text
print(message)

1,984,135と9,343,116の掛け算は、標準的な掛け算のアルゴリズムを使用して計算できます。2番目の数の各桁に最初の数をかけ、その結果を足し合わせましょう。

 1,984,135
x 9,343,116 
-----------
 11,904,810 (1,984,135 x 6)
 17,857,215 (1,984,135 x 9) 
 13,888,945 (1,984,135 x 7)
 3,968,270 (1,984,135 x 2)
 1,984,135 (1,984,135 x 1)
 17,857,215 (1,984,135 x 9)
 11,904,810 (1,984,135 x 6) 
 3,968,270 (1,984,135 x 2)
 17,857,215 (1,984,135 x 9)
 13,888,945 (1,984,135 x 7) 
 15,873,080 (1,984,135 x 8)
 5,952,405 (1,984,135 x 3)
----------- 
18,529,877,865,540

よって、1,984,135と9,343,116の掛け算の答えは18,529,877,865,540です。


In [6]:
answer = 1984135 * 9343116
print(f"{answer:,}")

18,538,003,464,660


クロードの答えは正解の0.01%以内でしたが、正確ではありませんでした。それを修正しましょう。まず、電卓関数を定義します。

In [5]:
def do_pairwise_arithmetic(num1, num2, operation):
 if operation == '+':
 return num1 + num2
 elif operation == "-":
 return num1 - num2 
 elif operation == "*":
 return num1 * num2
 elif operation == "/": 
 return num1 / num2
 else:
 return "エラー：サポートされていない演算です。"


次に、クロードに読ませるdocstringを書きます。

In [6]:
def construct_format_tool_for_claude_prompt(name, description, parameters):
 constructed_prompt = (
 "<tool_description>\n"
 f"<tool_name>{name}</tool_name>\n"
 "<description>\n"
 f"{description}\n"
 "</description>\n"
 "<parameters>\n"
 f"{construct_format_parameters_prompt(parameters)}\n"
 "</parameters>\n"
 "</tool_description>"
 )
 return constructed_prompt

tool_name = "calculator"
tool_description = """基本的な算術を行うための電卓関数です。
加算、減算、乗算をサポートしています。"""

def construct_format_parameters_prompt(parameters):
 constructed_prompt = "\n".join(f"<parameter>\n<name>{parameter['name']}</name>\n<type>{parameter['type']}</type>\n<description>{parameter['description']}</description>\n</parameter>" for parameter in parameters) 

 return constructed_prompt

parameters = [
 {
 "name": "first_operand",
 "type": "int",
 "description": "第1オペランド（演算子の前）"
 },
 {
 "name": "second_operand", 
 "type": "int",
 "description": "第2オペランド（演算子の後）"
 },
 {
 "name": "operator",
 "type": "str",
 "description": "実行する演算。+、-、*、/のいずれかでなければなりません"
 }
]
tool = construct_format_tool_for_claude_prompt(tool_name, tool_description, parameters)
print(tool)

<tool_description>
<tool_name>calculator</tool_name>
<description>
基本的な算術を行うための電卓関数です。
加算、減算、乗算をサポートしています。
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第1オペランド（演算子の前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第2オペランド（演算子の後）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>実行する演算。+、-、*、/のいずれかでなければなりません</description>
</parameter>
</parameters>
</tool_description>


この関数の説明を長いプロンプトテンプレートに挿入して、システムプロンプトを作成します。

In [7]:
def construct_tool_use_system_prompt(tools):
 tool_use_system_prompt = (
 "この環境では、ユーザーの質問に答えるための一連のツールにアクセスできます。\n"
 "\n"
 "次のように呼び出すことができます。\n"
 "<function_calls>\n"
 "<invoke>\n" 
 "<tool_name>$TOOL_NAME</tool_name>\n"
 "<parameters>\n"
 "<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>\n"
 "...\n"
 "</parameters>\n"
 "</invoke>\n"
 "</function_calls>\n"
 "\n"
 "利用可能なツールは次のとおりです。\n"
 "<tools>\n"
 + '\n'.join([tool for tool in tools]) +
 "\n</tools>"    )
    return tool_use_system_prompt

system_prompt = construct_tool_use_system_prompt([tool])
print(system_prompt)

この環境では、ユーザーの質問に答えるための一連のツールにアクセスできます。

次のように呼び出すことができます。
<function_calls>
<invoke> 
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

利用可能なツールは次のとおりです。
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
基本的な算術を行うための電卓関数です。
加算、減算、乗算をサポートしています。
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第1オペランド（演算子の前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第2オペランド（演算子の後）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>実行する演算。+、-、*、/のいずれかでなければなりません</description>
</parameter>
</parameters>
</tool_description>
</tools>


次に必要なのは、このプロンプトを使ってクロードを呼び出すためのハーネスを作ることだけです。まず、このシステムプロンプトを与えて、以前と同じ質問をした場合のクロードの出力を見てみましょう。

はい、それでは順を追って説明しましょう。
<function_calls>
<invoke>
<tool_name>calculator</tool_name>
<parameters>
<first_operand>1984135</first_operand>
<second_operand>9343116</second_operand>
<operator></operator>
</parameters>
</invoke>



クロードは関数を正しく呼び出すことができました。次に、パラメータを抽出して、do_pairwise_arithmetic関数に送ります。

18,538,003,464,660


完璧です！あとは戻り値をクロードに渡して、ユーザーに最終的な値を返すだけです。まずクロードが期待する形式でフォーマットします。

<function_results>
<result>
<tool_name>do_pairwise_arithmetic</tool_name>
<stdout>
18538003464660
</stdout>
</result>
</function_results>


次に、元のメッセージ、関数が呼び出されるまでのクロードの部分的なリターン、および関数の結果を組み合わせて、クロードに最終的な出力を生成するためのプロンプトを作成します。これを円滑にするためにassistantの役割でメッセージを事前に埋め込みます。

はい、それでは順を追って説明しましょう。
<function_calls>
<invoke>
<tool_name>calculator</tool_name>
<parameters>
<first_operand>1984135</first_operand>
<second_operand>9343116</second_operand>
<operator></operator>
</parameters>
</invoke>
</function_calls><function_results>
<result>
<tool_name>do_pairwise_arithmetic</tool_name>
<stdout>
18538003464660
</stdout>
</result>
</function_results>

したがって、1,984,135と9,343,116の掛け算の答えは18,538,003,464,660です。


成功です！ここで定義したプロンプトコンストラクタと関数呼び出しのパターンを使用して、独自の関数を実装できます。たとえば、検索、SQL、インターネットへの呼び出しなどです。最良の結果を得るには、こことanthropic-toolsリポジトリに示されている正確なシステムプロンプトのフォーマットと<function_calls>/<function_results>のフォーマットを使用してください。